In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
import csv
import os
from google.colab import drive
drive.mount('/content/drive', force_remount = True) # mount drive to pull files

Mounted at /content/drive


In [ ]:
# read in coordinates .csv from drive
coordinate_file = "" # your coordinate file
points = pd.read_csv(coordinate_file)

In [ ]:
def rotate_vector(vector, angle):
    """ Rotates a vector around the origin by `angle` radians.
    Args:
        vector: A numpy array of length 2 representing the vector to be rotated.
        angle: The angle in radians by which the vector should be rotated.
    Returns:
        A numpy array of length 2 representing the rotated vector.
    """
    x = np.array(vector[0]) * math.cos(angle) - np.array(vector[1]) * math.sin(angle)
    y = np.array(vector[0]) * math.sin(angle) + np.array(vector[1]) * math.cos(angle)

    return [x, y]

## Extract Angles and Create Plots/CSV Files

In [ ]:
def data_transformation(df, file_name):

  points = df

  # Filter data where time is less than or equal to 95 (case-specific)
  # points = df[df['index'] <= 2222]

  angleDeg = [] # initialize list of angles
  for i in range(len(points)):
    # define coordinates of three points
    nx, ny = points['nose'][i], points['nose.1'][i]
    dx, dy = points ['dorsalfin'][i], points ['dorsalfin.1'][i]
    cx, cy = points ['caudalfin'][i], points ['caudalfin.1'][i]

    # create vectors
    nd, nc = [(dx - nx), (dy - ny)], [(cx - nx), (cy - ny)] # nd, nc vectors

    # make unit vectors
    ndUnit, ncUnit = nd/np.linalg.norm(nd), nc/np.linalg.norm(nc) # unit vectors

    yNegUnit = [0,-1] # unit vector on the negative y axis

    # get the rotation angle (from nd to the <0, -1> vector)
    theta = np.arccos(np.dot(yNegUnit, ndUnit))
    if (ndUnit[0] > 0): theta = 2* math.pi - theta # angle clause if x-value of nd is negative (q3, q4)

    # rotate vectors
    ndTransf, ncTransf = rotate_vector(ndUnit, theta), rotate_vector(ncUnit, theta)

    # calculate angle
    angle = np.degrees(np.arccos(np.dot(ndTransf, ncTransf)))

    # left-right (-L, +R)
    if (ncTransf[0] < 0): angle = -angle # if the x-value of the transformed nc vector is -; make angle - (otherwise stay +)

    angleDeg.append(angle)

  # create dataframe of angles
  df_ofAngles = pd.DataFrame(angleDeg, columns=['angles'])

  # combine the dataframes vertically
  combined_df_with_angles = pd.concat([points, df_ofAngles], axis=1)

  ############### Data Cleaning

  # biological threshold
  for i, angle in enumerate(combined_df_with_angles['angles']):
    if angle > 45 or angle < -45:
      combined_df_with_angles.at[i, 'angles'] = np.nan


  # z-score for sine curve
  # calculate the interquartile range (IQR)
  Q1 = combined_df_with_angles['angles'].quantile(0.25)
  Q3 = combined_df_with_angles['angles'].quantile(0.75)
  IQR = Q3 - Q1

  # define the lower and upper bounds for outlier detection
  lower_bound = Q1 - 1.5 * IQR
  upper_bound = Q3 + 1.5 * IQR

  # find outliers in the 'y' column
  df_filtered = combined_df_with_angles[
      (combined_df_with_angles['angles'] >= lower_bound) &
      (combined_df_with_angles['angles'] <= upper_bound) &
      (combined_df_with_angles['angles'] <= 45) &
      (combined_df_with_angles['angles'] >= -45)
  ]

  ############### Plotting 

  # plot setup 
  plt.figure(figsize=(10, 5))
  plt.xlabel('Time (sec)')
  plt.ylabel('Angle (degrees)')
  plt.plot(df_filtered['index']/24, df_filtered['angles'], color = 'gray', zorder=1)
  plt.plot(df_filtered['index']/24, df_filtered['angles'], color = 'green', label='Filtered Data', marker='.', markersize = 10, alpha=0.5, zorder=1)
  # Plot horizontal dashed lines at 45 and -45 degrees
  plt.axhline(y=45, color='black', linestyle='--', label='45°')
  plt.axhline(y=-45, color='black', linestyle='--', label='-45°')

  ############### Saving data

  # Set the path to the CSV file in your Drive where you want the output csv file to be saved
  path = f'/content/drive/MyDrive/2022_2023_transformed/{file_name}.csv'
  new_path = os.path.splitext(path)[0] + '_transformed.csv'

  # Save the dataframe as a CSV file in your Drive
  df_filtered.to_csv(new_path, index=False) #can change index to true if want indexes saved with it

  # Save the plots to a folder in your Drive
  images_dir = f'/content/drive/MyDrive/2022_2023_vector_rotation_plots'
  plt.savefig(f'{images_dir}/{file_name}.png')